In [253]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

chrome_driver_path = "C:\Metis_Bootcamp\Regression_project\chromedriver.exe"

from bs4 import BeautifulSoup
import requests

import re

In [254]:
url = "https://www.cars.com/for-sale/searchresults.action/?page=1&perPage=100&rd=500&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125"

header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
}

response = requests.get(url, headers=header)

data = response.text
soup = BeautifulSoup(data, "html.parser")

In [255]:
all_page_links = []

for i in range(len(soup.find_all(class_="shop-srp-listings__listing-container"))):
    link = soup.find_all(class_="shop-srp-listings__listing-container")[i].find('a')['href']
    all_page_links.append(f"https://www.cars.com/{link}")
all_page_links

['https://www.cars.com//vehicledetail/detail/840802779/overview/',
 'https://www.cars.com//vehicledetail/detail/841973875/overview/',
 'https://www.cars.com//vehicledetail/detail/826215383/overview/',
 'https://www.cars.com//vehicledetail/detail/841257282/overview/',
 'https://www.cars.com//vehicledetail/detail/841376537/overview/',
 'https://www.cars.com//vehicledetail/detail/841120458/overview/',
 'https://www.cars.com//vehicledetail/detail/840921405/overview/',
 'https://www.cars.com//vehicledetail/detail/841020014/overview/',
 'https://www.cars.com//vehicledetail/detail/841566704/overview/',
 'https://www.cars.com//vehicledetail/detail/841376547/overview/',
 'https://www.cars.com//vehicledetail/detail/840996901/overview/',
 'https://www.cars.com//vehicledetail/detail/842319454/overview/',
 'https://www.cars.com//vehicledetail/detail/842881071/overview/',
 'https://www.cars.com//vehicledetail/detail/842284805/overview/',
 'https://www.cars.com//vehicledetail/detail/841830015/overvie

In [299]:
df_headers = ["Name", "Mileage", "Address", "Rating", "Fuel Type", "City MPG", 
           "Highway MPG", "Drivetrain", "Engine", "Exterior Color", "Interior Color", "Transmission", "Entertainment", "Safety", "Price"]



In [257]:
all_page_links[:10]

['https://www.cars.com//vehicledetail/detail/840802779/overview/',
 'https://www.cars.com//vehicledetail/detail/841973875/overview/',
 'https://www.cars.com//vehicledetail/detail/826215383/overview/',
 'https://www.cars.com//vehicledetail/detail/841257282/overview/',
 'https://www.cars.com//vehicledetail/detail/841376537/overview/',
 'https://www.cars.com//vehicledetail/detail/841120458/overview/',
 'https://www.cars.com//vehicledetail/detail/840921405/overview/',
 'https://www.cars.com//vehicledetail/detail/841020014/overview/',
 'https://www.cars.com//vehicledetail/detail/841566704/overview/',
 'https://www.cars.com//vehicledetail/detail/841376547/overview/']

In [300]:
def get_car_features(url):
    
    response = requests.get(url, headers=header)

    data = response.text
    
    soup = BeautifulSoup(data, "html.parser")
    
    basic_features = soup.find_all('li', class_='vdp-details-basics__item')
    basic_feature_list = []
    for i in range(len(basic_features)):
        basic_feature_list.append(soup.find_all('li', class_='vdp-details-basics__item')[i].text.strip().split(':'))

    extra_features = soup.find_all(class_='details-feature-list details-feature-list--normalized-features')
    extra_feature_list = []
    for i in range(len(extra_features)):
        extra_feature_list.append(soup.find_all(class_='details-feature-list details-feature-list--normalized-features')[i].text.strip().split('\n\n'))
        
    name = soup.find('h1', class_='cui-heading-2--secondary vehicle-info__title').text
    mileage = soup.find(class_='vdp-cap-price__mileage--mobile vehicle-info__mileage mileage_margin').text.split(" ")[0]
    address = soup.find(class_='get-directions-link seller-details-location__text').text.strip()
    rating = soup.find(class_='rating__link rating__link--has-reviews')
    fuel_type = -1
    city_MPG = -1
    highway_MPG = -1
    drivetrain = -1
    engine = -1
    exterior_color = -1
    interior_color = -1
    transmission = -1
    entertainment = -1
    safety = -1
    price = soup.find(class_='vehicle-info__price-display vehicle-info__price-display--dealer cui-heading-2').text

    for basic in basic_feature_list:
        if basic[0] == "Fuel Type":
            fuel_type = basic[1]
        if basic[0] == 'City MPG':
            city_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Highway MPG':
            highway_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Drivetrain':
            drivetrain = basic[1]
        if basic[0] == 'Engine':
            engine = basic[1]
        if basic[0] == 'Exterior Color':
            exterior_color = basic[1]
        if basic[0] == 'Interior Color':
            interior_color = basic[1]
        if basic[0] == 'Transmission':
            transmission = basic[1]

    for extra in extra_feature_list:
        if extra[0] == "Entertainment":
            entertainment = extra[1]
        if extra[0] == "Safety":
            safety = extra[1]
            
    car_dict = dict(zip(df_headers, [name,
                             mileage,
                             address,
                             rating,
                             fuel_type,
                             city_MPG,
                             highway_MPG,
                             drivetrain,
                             engine,
                             exterior_color,
                             interior_color,
                             transmission,
                             entertainment,
                             safety,
                             price]))

    return cars_list.append(car_dict)

In [295]:
cars_list = []

for url in all_page_links:
    try:
        get_car_features(url)
    except:
        print(url)

pd.DataFrame(cars_list)

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price
0,2006 Toyota Sequoia Limited,"216,369","Edmonds, WA 98026","[(3.9) , [20 Reviews]]",Gasoline,15,17,4WD,4.7L V8 32V MPFI DOHC,Gray,Tan,5-Speed Automatic,Premium Sound System,Brake Assist\nStability Control,"$11,999"
1,2016 Volkswagen Golf GTI SE,"47,303","Kirkland, WA 98033","[(4.7) , [195 Reviews]]",Gasoline,25,34,FWD,2.0L I4 16V GDI DOHC Turbo,Pure White,Titan Black,6-Speed Manual,Bluetooth,Backup Camera\nBrake Assist\nStability Control,"$18,998"
2,2018 Toyota Highlander XLE,"25,005","Lynnwood, WA 98036",None,Gasoline,20,26,AWD,3.5L V6 24V PDI DOHC,Ooh La La Rouge Mica,Black,8-Speed Automatic,Bluetooth,Backup Camera\nBrake Assist\nStability Control...,"$32,988"
3,2018 Lexus NX 300 Base,"27,185","Lynnwood, WA 98036","[(4.7) , [146 Reviews]]",Gasoline,22,28,AWD,2.0L I4 16V PDI DOHC Turbo,Caviar,Black,6-Speed Automatic,Bluetooth\nPremium Sound System,Backup Camera\nBrake Assist\nStability Control...,"$33,450"
4,2019 Lexus RX 350 350,"11,461","Lynnwood, WA 98036","[(4.7) , [146 Reviews]]",Gasoline,19,26,AWD,3.5L V6 24V PDI DOHC,Silver Lining Metallic,Black,8-Speed Automatic,Bluetooth,Backup Camera\nStability Control\nAutomatic Em...,"$46,995"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019 Ford F-150 XLT,"18,632","Pullman, WA 99163","[(4.9) , [182 Reviews]]",Gasoline,18,23,4WD,3.5L V6 24V PDI DOHC Twin Turbo,Magnetic Metallic,Medium Earth Gray,10-Speed Automatic,Bluetooth\nApple CarPlay/Android Auto,Backup Camera\nBrake Assist\nStability Control...,"$46,495"
96,2018 Ford F-150 Lariat,"47,385","Pullman, WA 99163","[(4.9) , [182 Reviews]]",Gasoline,19,24,4WD,3.5L V6 24V PDI DOHC Twin Turbo,Guard Metallic,Medium Earth Gray,10-Speed Automatic,Bluetooth\nApple CarPlay/Android Auto,Backup Camera\nBrake Assist\nStability Control...,"$50,495"
97,2020 Ford F-150 Lariat,"28,415","Pullman, WA 99163","[(4.9) , [182 Reviews]]",Gasoline,18,23,4WD,3.5L V6 24V PDI DOHC Twin Turbo,Star White,Black,10-Speed Automatic,Bluetooth\nApple CarPlay/Android Auto,Backup Camera\nBrake Assist\nStability Control...,"$53,500"
98,2015 Toyota Highlander Hybrid Limited Platinum,"92,578","Kalispell, MT 59901","[(4.9) , [75 Reviews]]",Hybrid,27,28,AWD,3.5L V6 24V MPFI DOHC Hybrid,Attitude Black Metallic,Almond,Automatic CVT,Bluetooth\nPremium Sound System,Backup Camera\nBrake Assist\nStability Control...,"$25,996"


        if basic[0] == "Fuel Type":
            fuel_type = basic[1]
        if basic[0] == 'City MPG':
            city_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Highway MPG':
            highway_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Drivetrain':
            drivetrain = basic[1]
        if basic[0] == 'Engine':
            engine = basic[1]
        if basic[0] == 'Exterior Color':
            exterior_color = basic[1]
        if basic[0] == 'Interior Color':
            interior_color = basic[1]
        if basic[0] == 'Transmission':
            transmission = basic[1]